# Visicule


## Data Preparation

In [ ]:
%run ../data/prepare-data.ipynb

## Implementation

In [ ]:
from mesonic.events import Event
from mesonic.backend.bases import EventHandler
from attrs import define

from typing import Callable, Dict, Iterable

@define(kw_only=True)
class CallableEvent(Event):

    callable: Callable
    kwargs: Dict
    rkwargs: Dict


class CallableEventHandler(EventHandler[CallableEvent]):
    def handle(
        self, time: float, events: Iterable[CallableEvent], reversed: bool, **kwargs
    ) -> None:
        """Handle events with the provided time and state.

        Parameters
        ----------
        time : float
            Timestamp provided for the events.
        events : Iterable[E]
            Events to be handled.
        reversed : bool
            Whether the events must be reversed or not.

        """
        for event in events:
            kwargs = event.kwargs if not reversed else event.rkwargs
            event.callable(**kwargs)

    def get_etype(self) -> type:
        """Get the type of Event this handler processes.

        Returns
        -------
        type
            type of Event this handler processes.

        """
        return CallableEvent

In [ ]:
import matplotlib.pyplot as plt

class UpdateableVline:
    def __init__(self, fig, ax, **kwargs):
        self.fig = fig
        self.ax = ax
        self.line = ax.axvline(0, **kwargs)

    def update(self, pos):
        self.line.set_xdata([pos, pos])
        self.fig.canvas.draw()
        self.fig.canvas.flush_events()
        return (self.line,)

## Usage / Demo Snippets

In [ ]:
import sonecules
from pya import Asig
from sonecules.bufferson import BasicAUD

In [ ]:
%matplotlib widget

In [ ]:
sonecules.startup()
context = sonecules.gcc()
pb = context.playback

In [ ]:
# add new event handler
callable_event_handler = CallableEventHandler()
context.processor.add_handler(callable_event_handler)

In [ ]:
# prepare plot
context.reset()

audification = BasicAUD.from_df(dataframes["eeg"], sr=256, columns=5)

fig, ax = plt.subplots(1)
audification.dasig.plot(ax=ax)
uvline = UpdateableVline(fig=fig, ax=ax)

rate = 20
audification.reschedule(at=0, rate=rate)
dur = audification.dasig.get_duration() / rate
for i in np.arange(0, dur, 1/25):
    kwargs = {"pos": i * rate}
    context.receive_event(CallableEvent(callable=uvline.update, kwargs=kwargs, rkwargs=kwargs), i)

In [ ]:
pb.start()